## Setup

In [ ]:
!nvidia-smi

In [ ]:
%pip install ultralytics

In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display,Image
from IPython import display
display.clear_output()
!yolo checks

## First Dataset - Fire-detection-v3-6 - Download

In [ ]:
%pip install roboflow

#Downloading the first dataset
from roboflow import Roboflow
rf = Roboflow(api_key="srgdsN7LdNoRBKPirPfv")
project = rf.workspace("touatimed2").project("fire-detection-v3-or0i1")
version = project.version(6)
dataset = version.download("yolov8")

## Second Dataset - Fire-Detection-1 - Download

In [ ]:
%pip install roboflow

#Downloading the second dataset
from roboflow import Roboflow
rf = Roboflow(api_key="srgdsN7LdNoRBKPirPfv")
project = rf.workspace("situational-awarnessinnovsense").project("fire-detection-ypseh")
version = project.version(1)
dataset = version.download("yolov8")

## Running Both Datasets for 100 Epochs with Stats and Saving Weights after 10 Epochs

In [ ]:
#Dataset - 1
!yolo task=detect mode=train model=yolov8n.pt data=Fire-detection-v3-6/data.yaml epochs=100 imgsz=640 verbose=True save_period=10 project=D1E100

In [ ]:
#Dataset - 2
!yolo task=detect mode=train model=yoloD1E100.pt data=Fire-Detection-1/data.yaml epochs=100 imgsz=640 verbose=True save_period=10 project=D2E100

## Transfer Learning - Adding Freeze for 100 Epochs



In [ ]:
#Dataset - 1 - 20 Epoch With Freeze
!yolo task=detect mode=train model=yolov8n.pt data=Fire-detection-v3-6/data.yaml epochs=20 freeze=10 imgsz=640 verbose=True save_period=10 project=D1E20F10

In [ ]:
#Dataset - 2 - 20 Epoch With Freeze
!yolo task=detect mode=train model=yolov8n.pt data=Fire-Detection-1/data.yaml epochs=20 freeze=10 imgsz=640 plots=true verbose=True save_period=10 project=D2E20F10

In [ ]:
#Dataset - 1- 50 Epoch With Freeze
!yolo task=detect mode=train model=yolov8n.pt data=Fire-detection-v3-6/data.yaml epochs=50 freeze=10 imgsz=640 plots=true verbose=True save_period=10 project=D1E50F10

In [ ]:
#Dataset - 2- 50 Epoch With Freeze
!yolo task=detect mode=train model=yolov8n.pt data=Fire-Detection-1/data.yaml epochs=50 freeze=10 imgsz=640 plots=true verbose=True save_period=10 project=D2E50F10

In [ ]:
#Dataset - 1 - 100 Epoch With Freeze
!yolo task=detect mode=train model=yolov8n.pt data=Fire-detection-v3-6/data.yaml epochs=100 freeze=10 imgsz=640 plots=true verbose=True save_period=1 project=D1E100F10

In [ ]:
#Dataset - 2 - 100 Epoch With Freeze
!yolo task=detect mode=train model=yolov8n.pt data=Fire-Detection-1/data.yaml epochs=100 freeze=10 imgsz=640 plots=true verbose=True save_period=1 project=D2E100F10

## K - Folds || Only the Second Dataset

In [ ]:
import datetime
import shutil
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
import yaml
from sklearn.model_selection import KFold

In [ ]:
dataset_path = Path("./Fire-detection-v3-6")  # replace to dataset as necessary
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'

In [ ]:
yaml_file = "Fire-detection-v3-6/data.yaml"  # replace Yaml as necessary
with open(yaml_file, "r", encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]
cls_idx = list(range(len(classes)))  # Create an index list based on the number of classes


In [ ]:
import pandas as pd

indx = [l.stem for l in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [ ]:
for label in labels:
    lbl_counter = Counter()

    with open(label, "r") as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(" ")[0])] += 1

    # Convert the Counter to a dictionary and then to a DataFrame row
    lbl_counter_dict = dict(lbl_counter)
    lbl_counter_df = pd.DataFrame([lbl_counter_dict], index=[label.stem])
    
    # Concatenate the new row to the main DataFrame
    labels_df = pd.concat([labels_df, lbl_counter_df], axis=0)

# Replace `nan` values with `0.0`
labels_df = labels_df.fillna(0.0)

print(labels_df)

In [ ]:
ksplit = 5 # 5 Splits 
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [ ]:
folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f"split_{idx}"].loc[labels_df.iloc[train].index] = "train"
    folds_df[f"split_{idx}"].loc[labels_df.iloc[val].index] = "val"

In [ ]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio

In [ ]:
import datetime

supported_extensions = [".jpg", ".jpeg", ".png"]

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / "images").rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f"{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )

In [ ]:
print(folds_df) # An image is used to train in 3 models and validate in the rest two as seen from printing

In [ ]:
import shutil
import logging
from pathlib import Path
import datetime
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Define base paths and other variables
dataset_path = Path("./Fire-detection-v3-6")
save_path = Path(dataset_path / f"{datetime.date.today().isoformat()}_5-Fold_Cross-val")
train_base_path = dataset_path / "train"
valid_base_path = dataset_path / "valid"

# Loop through supported extensions and gather image files from train directory
supported_extensions = [".jpg", ".jpeg", ".png"]
images = []

for ext in supported_extensions:
    images.extend(sorted((train_base_path / "images").rglob(f"*{ext}")))

# Ensure each split directory and subdirectories exist
for split_num in range(1, 6):
    for split_type in ["train", "val"]:
        (save_path / f"split_{split_num}" / split_type / "images").mkdir(parents=True, exist_ok=True)
        (save_path / f"split_{split_num}" / split_type / "labels").mkdir(parents=True, exist_ok=True)

# Assuming folds_df is already defined and loaded
for image in images:
    # Extract image stem to locate corresponding label
    image_stem = image.stem

    # Loop through folds_df to determine train/valid split for each image
    for split in folds_df.columns:
        k_split = folds_df.loc[image_stem, split]

        # Determine if it's train or valid split
        split_folder = "train" if k_split == "train" else "val"

        # Folder for the specific split (assuming `split` is in the format `split_1`, `split_2`, etc.)
        img_to_path = save_path / split / split_folder / "images"
        lbl_to_path = save_path / split / split_folder / "labels"

        # Ensure destination directories exist
        img_to_path.mkdir(parents=True, exist_ok=True)
        lbl_to_path.mkdir(parents=True, exist_ok=True)

        try:
            logging.debug(f"Copying {image} to {img_to_path / image.name}")
            shutil.copy(image, img_to_path / image.name)

            # Derive the label file path from the train directory
            label_file = train_base_path / "labels" / f"{image_stem}.txt"
            
            logging.debug(f"Copying {label_file} to {lbl_to_path / label_file.name}")
            shutil.copy(label_file, lbl_to_path / label_file.name)

        except Exception as e:
            logging.error(f"Error copying {image} or {label_file}: {e}")

        # Check if files exist after copy (optional)
        if not (img_to_path / image.name).exists():
            logging.warning(f"File {image} not copied to {img_to_path / image.name}")

        if not (lbl_to_path / label_file.name).exists():
            logging.warning(f"File {label_file} not copied to {lbl_to_path / label_file.name}")

# Now do the same for images in the valid directory
images_valid = []

for ext in supported_extensions:
    images_valid.extend(sorted((valid_base_path / "images").rglob(f"*{ext}")))

for image in images_valid:
    # Extract image stem to locate corresponding label
    image_stem = image.stem

    # Loop through folds_df to determine train/valid split for each image
    for split in folds_df.columns:
        k_split = folds_df.loc[image_stem, split]

        # Determine if it's train or valid split
        split_folder = "train" if k_split == "train" else "val"

        # Folder for the specific split (assuming `split` is in the format `split_1`, `split_2`, etc.)
        img_to_path = save_path / split / split_folder / "images"
        lbl_to_path = save_path / split / split_folder / "labels"

        # Ensure destination directories exist
        img_to_path.mkdir(parents=True, exist_ok=True)
        lbl_to_path.mkdir(parents=True, exist_ok=True)

        try:
            logging.debug(f"Copying {image} to {img_to_path / image.name}")
            shutil.copy(image, img_to_path / image.name)

            # Derive the label file path from the valid directory
            label_file = valid_base_path / "labels" / f"{image_stem}.txt"
            
            logging.debug(f"Copying {label_file} to {lbl_to_path / label_file.name}")
            shutil.copy(label_file, lbl_to_path / label_file.name)

        except Exception as e:
            logging.error(f"Error copying {image} or {label_file}: {e}")

        # Check if files exist after copy (optional)
        if not (img_to_path / image.name).exists():
            logging.warning(f"File {image} not copied to {img_to_path / image.name}")

        if not (lbl_to_path / label_file.name).exists():
            logging.warning(f"File {label_file} not copied to {lbl_to_path / label_file.name}")


In [ ]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

In [ ]:
for k in range(1, 6):  # Train 5 splits
    !yolo task=detect mode=train model=yolov8n.pt data=Fire-detection-v3-6/2024-06-28_5-Fold_Cross-val/split_{k}/split_{k}_dataset.yaml epochs=100 imgsz=640 verbose=True save_period=10 project=kfold_first batch=16

## Third Dataset - Fire-and-Smoke-Detection-1 - Download

In [ ]:
%pip install roboflow

#Dataset 3 

from roboflow import Roboflow
rf = Roboflow(api_key="4FZgOnJSlZTxvjwBw0mO")
project = rf.workspace("adib-ga0ow").project("fire-and-smoke-detection-jngig")
version = project.version(1)
dataset = version.download("yolov8")

## Third Dataset - Train for 270 Epoch

In [ ]:
# Dataset 3 train for 270 Epochs
!yolo task=detect mode=train model=yolov8n.pt data=Fire-and-Smoke-Detection-1/data.yaml epochs=275 imgsz=640 verbose=True save_period=10  batch=16 project=D3E275